# SQL Assignment — Solutions

**Prepared for:** Sanskriti Jaiswal

I used the assignment PDF you uploaded as the reference. fileciteturn0file0

This notebook contains SQL answers (create/alter queries, explanations, and example queries) for every question in the assignment. You can copy the SQL into your DB client (MySQL/Postgres/SQLite — small syntax differences noted where necessary).

## Question 1 — Create `employees` table

Create a table called `employees` with the following structure and constraints:

- `emp_id` (integer, NOT NULL, PRIMARY KEY)
- `emp_name` (text, NOT NULL)
- `age` (integer, CHECK >= 18)
- `email` (text, UNIQUE)
- `salary` (decimal, default 30000)

**SQL (PostgreSQL / MySQL compatible)**:

In [ ]:
-- PostgreSQL / MySQL (modern) version
CREATE TABLE employees (
    emp_id INTEGER NOT NULL PRIMARY KEY,
    emp_name TEXT NOT NULL,
    age INTEGER CHECK (age >= 18),
    email TEXT UNIQUE,
    salary DECIMAL(10,2) DEFAULT 30000.00
);

**Notes:**
- MySQL before 8.0 does not enforce simple `CHECK` constraints (they are parsed but ignored); use triggers or application logic for strict enforcement or use `age INT NOT NULL` with additional constraints.
- `salary` is DECIMAL(10,2) with default 30000.00.


## Question 2 — Purpose of constraints and examples

**Answer (short):** Constraints enforce rules on table columns to maintain data integrity and consistency. They prevent invalid or inconsistent data from being stored.

**Common constraints with examples:**

- `PRIMARY KEY` — uniquely identifies each row (implies NOT NULL and UNIQUE).
- `NOT NULL` — ensures a column always has a value.
- `UNIQUE` — ensures column values are unique across rows.
- `FOREIGN KEY` — enforces referential integrity between two tables.
- `CHECK` — enforces a boolean expression (e.g., `age >= 18`).
- `DEFAULT` — supplies a default value when none is provided.

**Why they help:**
- Prevent duplicate or missing essential data (e.g., duplicate user emails).
- Ensure relationships remain consistent (foreign keys prevent orphan records).
- Enforce business rules (salary ranges, minimum age, valid status values).


## Question 3 — Why use NOT NULL? Can primary key contain NULL?

**Answer:**
- `NOT NULL` is applied when you require a value for every row — e.g., a user's name, an order date.
- A `PRIMARY KEY` cannot contain NULL values because it must uniquely identify a row and NULL means unknown. Most RDBMS enforce that primary key columns are implicitly NOT NULL.

**Justification:** Allowing NULL in a primary key would break uniqueness and identity semantics; therefore, primary keys are always NOT NULL.


## Question 4 — Add or remove constraints on an existing table

**Adding a constraint (example):** add primary key, unique, default, or check.

```sql
-- Add UNIQUE constraint on email (MySQL / PostgreSQL)
ALTER TABLE employees
ADD CONSTRAINT employees_email_unique UNIQUE (email);

-- Add CHECK constraint (Postgres)
ALTER TABLE employees
ADD CONSTRAINT employees_age_check CHECK (age >= 18);
```

**Removing a constraint (example):**

```sql
-- PostgreSQL: drop constraint by name
ALTER TABLE employees
DROP CONSTRAINT employees_email_unique;

-- MySQL: if constraint created as INDEX/UNIQUE KEY
ALTER TABLE employees
DROP INDEX employees_email_unique; -- or DROP KEY <index_name>
```

**Notes:** Constraint names differ between DB engines — when adding a constraint give it a name so you can drop it easily.


## Question 5 — Consequences of violating constraints

If you try to insert/update/delete data that violates constraints, the database will reject the operation and return an error. This prevents bad data entering the system.

**Examples of errors:**

- `NOT NULL` violation: `ERROR: null value in column "emp_name" violates not-null constraint`
- `UNIQUE` violation: `ERROR: duplicate key value violates unique constraint "employees_email_unique"`
- `CHECK` violation: `ERROR: new row for relation "employees" violates check constraint "employees_age_check"`
- `FOREIGN KEY` violation: `ERROR: insert or update on table "orders" violates foreign key constraint "orders_customer_id_fkey"`

These errors stop the transaction unless handled; they ensure integrity but require the application to handle/validate data before attempting the operation.


## Question 6 — Modify existing `products` table

Original table:
```sql
CREATE TABLE products (
    product_id INT,
    product_name VARCHAR(50),
    price DECIMAL(10,2)
);
```
Realise: `product_id` should be a primary key and `price` should default to 50.00

**Add constraints using ALTER:**

```sql
-- Make product_id NOT NULL and add primary key
ALTER TABLE products
MODIFY COLUMN product_id INT NOT NULL; -- MySQL syntax (or ALTER COLUMN in Postgres)

ALTER TABLE products
ADD PRIMARY KEY (product_id);

-- Add default for price (Postgres / MySQL):
ALTER TABLE products
ALTER COLUMN price SET DEFAULT 50.00; -- Postgres

-- MySQL alternative for default:
ALTER TABLE products
MODIFY COLUMN price DECIMAL(10,2) DEFAULT 50.00;
```

**Alternate approach:** Create a new table with desired constraints and copy data across.


## Question 7 — INNER JOIN students and classes

Assuming tables `students(student_id, student_name, class_id)` and `classes(class_id, class_name)`

**Query:**
```sql
SELECT s.student_name, c.class_name
FROM students s
INNER JOIN classes c
  ON s.class_id = c.class_id;
```


## Question 8 — Show all orders ensuring all products listed (LEFT JOIN)

Assume tables:
- `orders(order_id, customer_id, product_id)`
- `customers(customer_id, customer_name)`
- `products(product_id, product_name)`

**Goal:** Ensure all products appear even if not ordered: use `LEFT JOIN` from products to orders.

```sql
SELECT o.order_id, c.customer_name, p.product_name
FROM products p
LEFT JOIN orders o ON p.product_id = o.product_id
LEFT JOIN customers c ON o.customer_id = c.customer_id;
```

This lists every product; `order_id` and `customer_name` will be NULL for products with no orders.


## Question 9 — Total sales amount for each product

Assume tables:
- `order_items(order_id, product_id, quantity, unit_price)`
- `products(product_id, product_name)`

**Query (INNER JOIN + SUM):**
```sql
SELECT p.product_id, p.product_name,
       SUM(oi.quantity * oi.unit_price) AS total_sales
FROM products p
INNER JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name;
```

If you want to show also products with zero sales, use `LEFT JOIN` and `COALESCE(SUM(...),0)`.

```sql
SELECT p.product_id, p.product_name,
       COALESCE(SUM(oi.quantity * oi.unit_price), 0) AS total_sales
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name;
```


## Question 10 — Display `order_id`, `customer_name`, and quantity ordered by each customer

Assume tables:
- `orders(order_id, customer_id)`
- `customers(customer_id, customer_name)`
- `order_items(order_item_id, order_id, product_id, quantity)`

**Query (join all three):**
```sql
SELECT o.order_id, c.customer_name, SUM(oi.quantity) AS total_quantity
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY o.order_id, c.customer_name;
```

This displays the total quantity of products per order (per customer for that order).


---
# SQL Commands & Sakila / Mavenmovies Questions

Below are solutions and sample queries referencing the Sakila-style schema (movie rental DB). The assignment mentioned a dataset `Mavenmovies.sql` — the queries below use typical Sakila table/column names. Adjust column names as required in your specific dump.


## SQL Commands — 1. Identify primary keys and foreign keys in `maven movies` DB

**Answer (general):**
- Primary keys: Usually `actor.actor_id`, `film.film_id`, `customer.customer_id`, `rental.rental_id`, `payment.payment_id`, `inventory.inventory_id`, `staff.staff_id`, etc.
- Foreign keys: `film_actor(actor_id, film_id)` references `actor(actor_id)` and `film(film_id)`; `rental.inventory_id` references `inventory.inventory_id`; `payment.customer_id` references `customer.customer_id`; `address.city_id` references `city.city_id`.

**Difference:** Primary key uniquely identifies rows in its table; foreign key enforces referential integrity referencing a primary (or unique) key in another table.


## SQL Commands — common SELECT queries (Sakila-like)

Below are concise SQL queries for the numbered tasks (2–13 etc) from the assignment. Replace table names if your dump differs.

**2. List all details of actors**
```sql
SELECT * FROM actor;
```

**3. List all customer information**
```sql
SELECT * FROM customer;
```

**4. List different countries**
```sql
SELECT DISTINCT country FROM country; -- or SELECT DISTINCT country FROM city JOIN country ...
```

**5. Display all active customers**
```sql
SELECT * FROM customer WHERE active = 1; -- or active = 'Y' depending on schema
```

**6. List of all rental IDs for customer with ID 1**
```sql
SELECT rental_id FROM rental WHERE customer_id = 1;
```

**7. Films whose rental_duration > 5**
```sql
SELECT * FROM film WHERE rental_duration > 5;
```

**8. Total number of films whose replacement_cost > 15 AND < 20**
```sql
SELECT COUNT(*) FROM film WHERE replacement_cost > 15 AND replacement_cost < 20;
```

**9. Count of unique first names of actors**
```sql
SELECT COUNT(DISTINCT first_name) FROM actor;
```

**10. First 10 records from customer table**
```sql
SELECT * FROM customer ORDER BY customer_id LIMIT 10;
```

**11. First 3 records where first name starts with 'b'**
```sql
SELECT * FROM customer WHERE first_name LIKE 'b%' ORDER BY customer_id LIMIT 3;
```

**12. Names of first 5 movies rated 'G'**
```sql
SELECT title FROM film WHERE rating = 'G' LIMIT 5;
```

**13. Find customers whose first name starts with 'a'**
```sql
SELECT * FROM customer WHERE first_name LIKE 'a%';
```

**14. First name ends with 'a'**
```sql
SELECT * FROM customer WHERE first_name LIKE '%a';
```

**15. First 4 cities which start and end with 'a'**
```sql
SELECT city FROM city WHERE city LIKE 'a%' AND city LIKE '%a' LIMIT 4;
```

**16. First name has 'NI' in any position**
```sql
SELECT * FROM customer WHERE first_name LIKE '%NI%'; -- case-sensitive in some DBs
```

**17. First name has 'r' in second position**
```sql
SELECT * FROM customer WHERE first_name LIKE '_r%';
```

**18. First name starts with 'a' AND length >= 5**
```sql
SELECT * FROM customer WHERE first_name LIKE 'a%' AND CHAR_LENGTH(first_name) >= 5;
```

**19. First name starts with 'a' and ends with 'o'**
```sql
SELECT * FROM customer WHERE first_name LIKE 'a%o';
```

**20. Films with rating in ('PG','PG-13')**
```sql
SELECT * FROM film WHERE rating IN ('PG','PG-13');
```

**21. Films with length between 50 and 100**
```sql
SELECT * FROM film WHERE length BETWEEN 50 AND 100;
```

**22. Top 50 actors (limit)**
```sql
SELECT * FROM actor ORDER BY actor_id LIMIT 50;
```

**23. Distinct film ids from inventory table**
```sql
SELECT DISTINCT film_id FROM inventory;
```


## Functions — Basic aggregate and string functions

**Question 1: Total number of rentals**
```sql
SELECT COUNT(*) AS total_rentals FROM rental;
```

**Question 2: Average rental duration (days)**
```sql
SELECT AVG(rental_duration) AS avg_rental_duration FROM film; -- or from rental if rental_duration stored there
```

**Question 3: Display first and last name of customers in uppercase**
```sql
SELECT UPPER(first_name) AS first_name_up, UPPER(last_name) AS last_name_up FROM customer;
```

**Question 4: Extract month from rental date alongside rental ID**
```sql
SELECT rental_id, EXTRACT(MONTH FROM rental_date) AS rental_month FROM rental; -- Postgres
-- MySQL: SELECT rental_id, MONTH(rental_date) as rental_month FROM rental;
```


## GROUP BY queries (Q5-Q8)

**Q5: Count of rentals for each customer**
```sql
SELECT customer_id, COUNT(*) AS rentals_count
FROM rental
GROUP BY customer_id;
```

**Q6: Total revenue generated by each store**
```sql
SELECT store_id, SUM(amount) AS total_revenue
FROM payment
GROUP BY store_id;
```

**Q7: Total number of rentals for each category**
```sql
SELECT fc.category_id, c.name AS category_name, COUNT(*) AS rental_count
FROM film_category fc
JOIN film f ON fc.film_id = f.film_id
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY fc.category_id, c.name;
```

**Q8: Average rental rate of movies in each language**
```sql
SELECT l.language_id, l.name AS language_name, AVG(f.rental_rate) AS avg_rental_rate
FROM film f
JOIN language l ON f.language_id = l.language_id
GROUP BY l.language_id, l.name;
```


## Joins — Q9 to Q12

**Q9: title of movie, customer's first name & last name who rented it**
```sql
SELECT f.title, cu.first_name, cu.last_name
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN customer cu ON r.customer_id = cu.customer_id;
```

**Q10: Actors in film 'Gone with the Wind'**
```sql
SELECT a.first_name, a.last_name
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.title = 'Gone with the Wind';
```

**Q11: Customer names with total amount spent on rentals**
```sql
SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_spent
FROM customer c
JOIN payment p ON c.customer_id = p.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY total_spent DESC;
```

**Q12: Titles of movies rented by each customer in 'London'**
```sql
SELECT c.customer_id, c.first_name, c.last_name, f.title
FROM customer c
JOIN address a ON c.address_id = a.address_id
JOIN city ci ON a.city_id = ci.city_id
JOIN rental r ON c.customer_id = r.customer_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE ci.city = 'London'
GROUP BY c.customer_id, c.first_name, c.last_name, f.title;
```


## Advanced Joins & GROUP BY — Top rented & customers renting from both stores

**Q13: Top 5 rented movies with counts**
```sql
SELECT f.title, COUNT(*) AS times_rented
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id, f.title
ORDER BY times_rented DESC
LIMIT 5;
```

**Q14: Customers who have rented movies from both stores (1 and 2)**
```sql
SELECT r.customer_id
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
GROUP BY r.customer_id
HAVING SUM(CASE WHEN i.store_id = 1 THEN 1 ELSE 0 END) > 0
   AND SUM(CASE WHEN i.store_id = 2 THEN 1 ELSE 0 END) > 0;
```


## Window functions examples

**1. Rank customers based on total amount spent**
```sql
SELECT customer_id, first_name, last_name, total_spent,
       RANK() OVER (ORDER BY total_spent DESC) AS rank_spent
FROM (
  SELECT c.customer_id, c.first_name, c.last_name, SUM(p.amount) AS total_spent
  FROM customer c
  JOIN payment p ON c.customer_id = p.customer_id
  GROUP BY c.customer_id, c.first_name, c.last_name
) t;
```

**2. Cumulative revenue generated by each film over time** (example)
```sql
SELECT r.rental_date, f.film_id, f.title, SUM(p.amount) OVER (PARTITION BY f.film_id ORDER BY r.rental_date) AS cumulative_revenue
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN payment p ON r.rental_id = p.rental_id;
```

**3. Average rental duration by film using window**
```sql
SELECT film_id, title, AVG(rental_duration) OVER (PARTITION BY film_id) AS avg_duration
FROM film;
```

**4. Top 3 films per category by rental count**
```sql
SELECT category_id, title, times_rented
FROM (
  SELECT c.category_id, f.title,
         COUNT(r.rental_id) AS times_rented,
         RANK() OVER (PARTITION BY c.category_id ORDER BY COUNT(r.rental_id) DESC) AS rk
  FROM film_category c
  JOIN film f ON c.film_id = f.film_id
  JOIN inventory i ON f.film_id = i.film_id
  JOIN rental r ON i.inventory_id = r.inventory_id
  GROUP BY c.category_id, f.title
) t
WHERE rk <= 3;
```


## Normalization & CTE

**1NF / 2NF / 3NF examples (short answers):**

- **1NF violation example:** A `customers` table that stores multiple phone numbers in a single column `phone_numbers` (comma-separated). To normalize to 1NF, create a separate `customer_phone` table with `customer_id, phone_number` rows.

- **2NF:** Ensure every non-key attribute is fully dependent on the whole primary key (relevant for composite keys). If a table `order_items(order_id, product_id, product_name, price)` has `product_name` dependent only on `product_id`, move product data to `products` table.

- **3NF:** Remove transitive dependencies. If `employee(emp_id, dept_id, dept_name)` — `dept_name` depends on `dept_id` not directly on `emp_id`. Move departments to `department(dept_id, dept_name)` and reference `dept_id`.

**Normalization process example:** Start with unnormalized table -> separate repeating groups -> remove partial dependencies -> remove transitive dependencies.

**CTE Basics — sample queries:**

- Distinct list of actor names and number of films:
```sql
WITH actor_counts AS (
  SELECT actor_id, first_name, last_name
  FROM actor
)
SELECT a.first_name, a.last_name, COUNT(fa.film_id) AS films_count
FROM actor a
LEFT JOIN film_actor fa ON a.actor_id = fa.actor_id
GROUP BY a.first_name, a.last_name;
```

- CTE combining film and language:
```sql
WITH film_lang AS (
  SELECT f.film_id, f.title, l.name AS language_name, f.rental_rate
  FROM film f
  JOIN language l ON f.language_id = l.language_id
)
SELECT * FROM film_lang;
```

- CTE for total revenue per customer:
```sql
WITH customer_revenue AS (
  SELECT customer_id, SUM(amount) AS total_revenue
  FROM payment
  GROUP BY customer_id
)
SELECT c.customer_id, c.first_name, c.last_name, cr.total_revenue
FROM customer_revenue cr
JOIN customer c ON cr.customer_id = c.customer_id;
```

- CTE with window functions (rank films by rental duration):
```sql
WITH film_avg AS (
  SELECT film_id, title, rental_duration
  FROM film
)
SELECT film_id, title, rental_duration,
       RANK() OVER (ORDER BY rental_duration DESC) AS rank_duration
FROM film_avg;
```

- CTE to list customers with > 2 rentals:
```sql
WITH frequent_customers AS (
  SELECT customer_id, COUNT(*) AS rentals_count
  FROM rental
  GROUP BY customer_id
  HAVING COUNT(*) > 2
)
SELECT fc.customer_id, c.first_name, c.last_name, fc.rentals_count
FROM frequent_customers fc
JOIN customer c ON fc.customer_id = c.customer_id;
```

- Recursive CTE example to find staff reporting chain (Postgres / SQL Server):
```sql
WITH RECURSIVE reports AS (
  SELECT staff_id, first_name, last_name, reports_to
  FROM staff
  WHERE staff_id = 2 -- starting manager
  UNION ALL
  SELECT s.staff_id, s.first_name, s.last_name, s.reports_to
  FROM staff s
  JOIN reports r ON s.reports_to = r.staff_id
)
SELECT * FROM reports;
```


----

### Notebook saved
The full notebook with SQL statements and explanations has been saved to `/mnt/data/sql_assignment_solution.ipynb`.

You can download it using the link below.

